In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
from folium import GeoJson, GeoJsonTooltip
from matplotlib.patches import Patch
import warnings
warnings.filterwarnings('ignore')


# 1.2 Carga de datos desde GitHub


# 1.3 Primer vistazo
print("Shape del dataset:", df.shape)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
import requests
from folium.features import GeoJson, GeoJsonTooltip
from matplotlib.patches import Patch
import warnings
import os
warnings.filterwarnings('ignore')

# Configuraciones globales
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

def cargar_datos():
    """
    Carga y realiza la limpieza básica de los datos de beneficiarios
    """
    print("Cargando datos de beneficiarios...")
    
    url = "https://raw.githubusercontent.com/Emma-Ok/BootcampTalentoTech/main/beneficiarios.csv"

    df = pd.read_csv(url, delimiter=';', encoding='utf-8-sig')
    print(f"Dataset cargado: {len(df)} registros con {df.columns.shape[0]} variables")
    
    # Mostrar las primeras 5 líneas del DataFrame
    print("\nPrimeras 5 filas del DataFrame:")
    print(df.head())
    
    # Limpieza inicial (para quitar espacios en nombres de columnas)
    df.columns = df.columns.str.strip()
    
    # Eliminar filas con datos faltantes en columnas clave
    df = df.dropna(subset=['EDAD', 'PLATAFORMA_EDUCATIVA'])
    
    # Asegurar que la edad sea tipo entero
    df['EDAD'] = df['EDAD'].astype(int)
    
    # Normalizar plataforma educativa
    df['PLATAFORMA_EDUCATIVA'] = df['PLATAFORMA_EDUCATIVA'].str.strip().str.upper()
    
    # Normalizar nombres de departamentos
    df['DEPARTAMENTO'] = df['DEPARTAMENTO'].str.upper().str.strip()
    
    print(f"Datos procesados: {df.shape[0]} registros válidos")
    
    return df

def crear_grupos_etarios(df):
    """
    Crea grupos etarios para facilitar el análisis
    """
    # Crear grupos etarios con etiquetas personalizadas
    bins = [0, 25, 35, 45, 55, 100]
    labels = ['18-25', '26-35', '36-45', '46-55', '56+']
    df['GRUPO_ETARIO'] = pd.cut(df['EDAD'], bins=bins, labels=labels, right=True)
    
    return df

def grafico_departamentos(df):
    """
    Crea un gráfico de barras de beneficiarios por departamento
    """
    print("\nGenerando gráfico de beneficiarios por departamento...")
    
    # Configuración de estilo
    plt.style.use('classic')
    sns.set(rc={'axes.facecolor':'#f8f9fa', 'figure.facecolor':'white'})
    
    # Datos
    departamentos = df['DEPARTAMENTO'].value_counts().index.tolist()
    cantidades = df['DEPARTAMENTO'].value_counts().values.tolist()
    promedio = np.mean(cantidades)
    max_cantidad = max(cantidades)
    
    # Definir colores por categoría
    color_max = '#FF6B6B'  # Color para el máximo
    color_arriba_prom = '#4C72B0'  # Color para arriba del promedio
    color_abajo_prom = '#55A868'  # Color para abajo del promedio
    
    # Crear lista de colores según categorías
    colores = []
    for cantidad in cantidades:
        if cantidad == max_cantidad:
            colores.append(color_max)
        elif cantidad > promedio:
            colores.append(color_arriba_prom)
        else:
            colores.append(color_abajo_prom)
    
    # Crear figura
    plt.figure(figsize=(12, 7))
    ax = sns.barplot(x=departamentos, y=cantidades, palette=colores,
                     edgecolor='black', linewidth=0.8, alpha=0.85)
    
    # --- MEJORAS VISUALES ---
    # Título y etiquetas
    plt.title('Distribución de Beneficiarios por Departamento\n',
              fontsize=16, fontweight='bold', pad=20, color='#333333')
    plt.xlabel('Departamento', fontsize=12, labelpad=10, color='#555555')
    plt.ylabel('Número de Beneficiarios', fontsize=12, labelpad=10, color='#555555')
    
    # Rotación de etiquetas
    plt.xticks(rotation=45, ha='right', fontsize=10, color='#555555')
    plt.yticks(color='#555555')
    
    # Añadir valores en las barras
    for i, v in enumerate(cantidades):
        ax.text(i, v + (0.02*max_cantidad), str(v),
                ha='center',
                fontsize=10,
                fontweight='bold',
                color='#333333')
    
    # Línea horizontal de referencia mejorada
    plt.axhline(y=promedio,
                color='#DE8F05',
                linestyle='--',
                linewidth=2,
                alpha=0.8,
                label=f'Promedio: {int(promedio):,}',
                zorder=0)
    
    # Sombreado del área alrededor del promedio
    plt.fill_between(x=ax.get_xlim(),
                    y1=promedio*0.95,
                    y2=promedio*1.05,
                    color='#DE8F05',
                    alpha=0.1,
                    zorder=-1)
    
    # Leyenda personalizada
    legend_elements = [
        Patch(facecolor=color_max, label='Mayor cantidad'),
        Patch(facecolor=color_arriba_prom, label='Arriba del promedio'),
        Patch(facecolor=color_abajo_prom, label='Abajo del promedio'),
        plt.Line2D([0], [0],
                   color='#DE8F05',
                   linestyle='--',
                   linewidth=2,
                   alpha=0.8,
                   label=f'Promedio: {int(promedio):,}')
    ]
    plt.legend(handles=legend_elements, frameon=True, framealpha=1)
    
    # Ajustar límites del eje Y
    plt.ylim(0, max(cantidades) * 1.18)
    
    # Eliminar bordes innecesarios
    sns.despine(left=True, bottom=True)
    
    # Guardar gráfico
    plt.tight_layout()
    plt.savefig('graficos/beneficiarios_por_departamento.png', dpi=300)
    plt.close()
    
    print("Gráfico guardado: graficos/beneficiarios_por_departamento.png")

def grafico_genero(df):
    """
    Crea un gráfico de dona mostrando la distribución por género
    """
    print("\nGenerando gráfico de distribución por género...")
    
    # Configuración de estilo profesional
    plt.style.use('seaborn-v0_8-whitegrid')
    sns.set_palette("pastel")
    
    # Datos
    gender_counts = df['GENERO'].value_counts()
    labels = gender_counts.index.tolist()
    sizes = gender_counts.values.tolist()
    
    # Crear figura con tamaño adecuado
    plt.figure(figsize=(8, 8), facecolor='white')
    
    # Función para formato de porcentaje corregida
    def format_autopct(pct, sizes):
        total = sum(sizes)
        val = int(round(pct*total/100))
        return f'{pct:.1f}%\n({val:,})'
    
    # Gráfico de dona profesional
    plt.pie(sizes,
            labels=labels,
            autopct=lambda p: format_autopct(p, sizes),
            startangle=90,
            wedgeprops={'linewidth': 1, 'edgecolor': 'white'},
            textprops={'fontsize': 12, 'color': 'black', 'fontweight': 'bold'},
            colors=['#3498db', '#e91e63'],
            pctdistance=0.85)
    
    # Añadir círculo central para convertir en dona
    plt.gca().add_artist(plt.Circle((0,0), 0.70, fc='white'))
    
    # Título y anotaciones
    plt.title('Distribución por Género\n',
              fontsize=16, fontweight='bold', pad=20, color='#333333')
    
    # Leyenda profesional
    plt.legend(labels,
               title="Género",
               loc="center left",
               bbox_to_anchor=(1, 0.5),
               frameon=True,
               shadow=True)
    
    # Añadir anotación con total
    total = sum(sizes)
    plt.text(0, 0, f'Total\n{total:,}',
             ha='center',
             va='center',
             fontsize=12,
             fontweight='bold',
             color='black')
    
    # Eliminar eje Y
    plt.ylabel('')
    
    # Guardar gráfico
    plt.tight_layout()
    plt.savefig('graficos/distribucion_genero.png', dpi=300)
    plt.close()
    
    print("Gráfico guardado: graficos/distribucion_genero.png")

def grafico_edad(df):
    """
    Crea un histograma de la distribución de edades
    """
    print("\nGenerando histograma de distribución de edades...")
    
    sns.set_style('whitegrid')
    
    plt.figure(figsize=(12, 6))
    sns.histplot(
        data=df,
        x='EDAD',
        bins=10,
        kde=True,
        color='darkblue',
        edgecolor='black',
        alpha=0.7
    )
    
    # Media y mediana
    media = df['EDAD'].mean()
    mediana = df['EDAD'].median()
    plt.axvline(media, color='red', linestyle='--', linewidth=2, label=f'Media: {media:.1f}')
    plt.axvline(mediana, color='orange', linestyle=':', linewidth=2, label=f'Mediana: {mediana:.1f}')
    plt.legend()
    
    # Etiquetas
    plt.title('📊 Distribución de Edades de los Beneficiarios', fontsize=16, fontweight='bold')
    plt.xlabel('Edad', fontsize=12)
    plt.ylabel('Cantidad de personas', fontsize=12)
    
    # Guías y bordes
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    sns.despine()
    
    # Guardar gráfico
    plt.tight_layout()
    plt.savefig('graficos/distribucion_edades.png', dpi=300)
    plt.close()
    
    print("Gráfico guardado: graficos/distribucion_edades.png")

def grafico_plataformas(df):
    """
    Crea un gráfico de barras de las plataformas educativas
    """
    print("\nGenerando gráfico de plataformas educativas...")
    
    # Limpiar y contar plataformas
    conteo = df['PLATAFORMA_EDUCATIVA'].value_counts()
    total = conteo.sum()
    
    # Crear DataFrame
    df_conteo = conteo.reset_index()
    df_conteo.columns = ['Plataforma', 'Cantidad']
    df_conteo['Porcentaje'] = (df_conteo['Cantidad'] / total * 100).round(1)
    
    # Asignar colores tipo semáforo según % y uso
    colores = []
    max_val = df_conteo['Cantidad'].max()
    
    for _, row in df_conteo.iterrows():
        if row['Cantidad'] == max_val:
            colores.append('#e74c3c')  # rojo
        elif row['Porcentaje'] < 10:
            colores.append('#2ecc71')  # verde
        else:
            colores.append('#f39c12')  # naranja
    
    # Crear figura
    plt.figure(figsize=(14, 7))
    bars = sns.barplot(
        data=df_conteo,
        x='Plataforma',
        y='Cantidad',
        palette=colores,
        edgecolor='black'
    )
    
    # Etiquetas sobre las barras
    for i, row in df_conteo.iterrows():
        plt.text(
            i,
            row['Cantidad'] + total * 0.01,
            f"{row['Cantidad']} ({row['Porcentaje']}%)",
            ha='center',
            va='bottom',
            fontsize=9,
            fontweight='bold'
        )
    
    # Leyenda manual
    leyenda = [
        Patch(color='#e74c3c', label=' Plataforma más usada'),
        Patch(color='#f39c12', label=' Uso intermedio (≥10%)'),
        Patch(color='#2ecc71', label=' Menos del 10% del total')
    ]
    plt.legend(handles=leyenda, title='Leyenda', loc='upper right')
    
    # Estética
    plt.title('Uso de Plataformas Educativas', fontsize=16, fontweight='bold')
    plt.xlabel('Plataforma', fontsize=12)
    plt.ylabel('Cantidad de Usuarios', fontsize=12)
    plt.xticks(rotation=30, ha='right', fontsize=10)
    sns.despine()
    
    # Guardar gráfico
    plt.tight_layout()
    plt.savefig('graficos/uso_plataformas.png', dpi=300)
    plt.close()
    
    print("Gráfico guardado: graficos/uso_plataformas.png")

def grafico_plataformas_genero(df):
    """
    Crea un heatmap de uso de plataformas por género
    """
    print("\nGenerando heatmap de plataformas por género...")
    
    # Análisis cruzado: Plataforma por género
    cross_gen_plat = pd.crosstab(df['PLATAFORMA_EDUCATIVA'], df['GENERO'])
    
    # Ordenar plataformas por uso total (de mayor a menor)
    orden = cross_gen_plat.sum(axis=1).sort_values(ascending=False).index
    cross_gen_plat_sorted = cross_gen_plat.loc[orden]
    
    # Calcular porcentajes por fila
    cross_pct = cross_gen_plat_sorted.div(cross_gen_plat_sorted.sum(axis=1), axis=0) * 100
    
    # Etiquetas combinadas: cantidad + porcentaje
    labels = cross_gen_plat_sorted.astype(str) + "\n(" + cross_pct.round(1).astype(str) + "%)"
    
    # Configuración general de estilo
    sns.set_style("white")
    sns.set(font_scale=1.0)
    plt.rcParams['font.family'] = 'sans-serif'
    
    # Crear figura
    plt.figure(figsize=(14, 8))
    
    # Crear heatmap
    ax = sns.heatmap(
        cross_gen_plat_sorted,
        annot=labels,
        fmt='',
        cmap='rocket_r',
        linewidths=0.6,
        linecolor='white',
        cbar_kws={'label': 'Cantidad de usuarios'}
    )
    
    # Títulos y etiquetas
    plt.title('Distribución del Uso de Plataformas Educativas por Género',
              fontsize=16, fontweight='bold', loc='left', pad=15)
    plt.xlabel('Género', fontsize=13, fontweight='bold', labelpad=10)
    plt.ylabel('Plataforma Educativa', fontsize=13, fontweight='bold', labelpad=10)
    
    # Ajustes de ticks
    plt.xticks(rotation=0, fontsize=11)
    plt.yticks(rotation=0, fontsize=10)
    
    # Guardar gráfico
    plt.tight_layout()
    plt.savefig('graficos/plataformas_genero.png', dpi=300)
    plt.close()
    
    print("Gráfico guardado: graficos/plataformas_genero.png")

def grafico_plataformas_edad(df):
    """
    Crea un heatmap de uso de plataformas por grupo etario
    """
    print("\nGenerando heatmap de plataformas por grupo etario...")
    
    # Tabla cruzada
    cross = pd.crosstab(df['PLATAFORMA_EDUCATIVA'], df['GRUPO_ETARIO'])
    
    # Ordenar plataformas de mayor a menor uso total
    platform_order = cross.sum(axis=1).sort_values(ascending=False).index
    cross = cross.loc[platform_order]
    
    # Calcular porcentajes por fila
    cross_pct = cross.div(cross.sum(axis=1), axis=0) * 100
    
    # Crear anotaciones con valor + porcentaje
    annot = cross.astype(str) + '\n' + cross_pct.round(1).astype(str) + '%'
    
    # Visualización
    plt.figure(figsize=(14, 7))
    sns.set_style('whitegrid')
    cmap = sns.light_palette("navy", as_cmap=True)
    
    sns.heatmap(
        cross,
        annot=annot,
        fmt='',
        cmap=cmap,
        linewidths=0.5,
        linecolor='white',
        cbar_kws={'label': 'Cantidad de usuarios'},
        annot_kws={"size": 9}
    )
    
    plt.title('🧠 Uso de Plataformas por Grupo Etario', fontsize=18, fontweight='bold', loc='left')
    plt.xlabel('Grupo Etario', fontsize=13, fontweight='bold')
    plt.ylabel('Plataforma Educativa (ordenadas de mayor a menor uso)', fontsize=13, fontweight='bold')
    plt.xticks(rotation=30, ha='right', fontsize=11)
    plt.yticks(fontsize=11)
    
    # Guardar gráfico
    plt.tight_layout()
    plt.savefig('graficos/plataformas_grupo_etario.png', dpi=300)
    plt.close()
    
    print("Gráfico guardado: graficos/plataformas_grupo_etario.png")

def grafico_grupos_plataformas_barras(df):
    """
    Crea un gráfico de barras agrupadas por grupo etario y plataforma
    """
    print("\nGenerando gráfico de barras de grupos etarios y plataformas...")
    
    # Conteo de combinaciones
    conteo = df.groupby(['GRUPO_ETARIO', 'PLATAFORMA_EDUCATIVA']).size().unstack(fill_value=0)
    
    # Paleta personalizada
    custom_palette = sns.color_palette("Set2", n_colors=conteo.columns.shape[0])
    
    # Crear gráfico
    plt.figure(figsize=(14, 7))
    ax = conteo.plot(kind='bar', stacked=False, figsize=(14, 7), color=custom_palette, edgecolor='black')
    
    # Agregar etiquetas de valor
    for bars in ax.containers:
        ax.bar_label(
            bars,
            label_type='edge',
            fontsize=9,
            padding=3,
            color='black',
            weight='bold'
        )
    
    # Estética
    plt.title('🎓 Distribución de Plataformas por Grupo Etario', fontsize=16, fontweight='bold')
    plt.ylabel('Número de Beneficiarios', fontsize=12)
    plt.xlabel('Grupo Etario', fontsize=12)
    plt.xticks(rotation=0, fontsize=10)
    plt.yticks(fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.legend(title='Plataforma Educativa', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9, title_fontsize=10)
    
    # Guardar gráfico
    plt.tight_layout()
    plt.savefig('graficos/distribucion_plataformas_grupo_etario.png', dpi=300)
    plt.close()
    
    print("Gráfico guardado: graficos/distribucion_plataformas_grupo_etario.png")

def analisis_estadistico(df):
    """
    Genera un resumen estadístico detallado
    """
    print("\n" + "="*60)
    print("ESTADÍSTICAS RESUMEN Y INSIGHTS CLAVE")
    print("="*60)

    # Estadísticas generales
    print(f"\n📊 ESTADÍSTICAS GENERALES:")
    print(f"• Total de beneficiarios: {len(df):,}")
    print(f"• Edad promedio: {df['EDAD'].mean():.1f} años")
    print(f"• Edad mediana: {df['EDAD'].median():.1f} años")
    print(f"• Desviación estándar edad: {df['EDAD'].std():.1f} años")

    # Plataforma más popular
    most_popular = df['PLATAFORMA_EDUCATIVA'].value_counts()
    print(f"\n🥇 PLATAFORMA MÁS POPULAR:")
    print(f"• {most_popular.index[0]}: {most_popular.iloc[0]:,} usuarios ({most_popular.iloc[0]/len(df)*100:.1f}%)")

    # Distribución por género
    gender_dist = df['GENERO'].value_counts()
    print(f"\n👥 DISTRIBUCIÓN POR GÉNERO:")
    for gender, count in gender_dist.items():
        print(f"• {gender}: {count:,} ({count/len(df)*100:.1f}%)")
    
    # Grupos de edad más activos
    age_groups = df['GRUPO_ETARIO'].value_counts()
    print(f"\n📈 GRUPOS DE EDAD MÁS ACTIVOS:")
    for age_group, count in age_groups.head(3).items():
        print(f"• {age_group}: {count:,} usuarios ({count/len(df)*100:.1f}%)")

    # Correlaciones de edad por plataforma
    print(f"\n🎯 EDAD PROMEDIO POR PLATAFORMA:")
    age_by_platform = df.groupby('PLATAFORMA_EDUCATIVA')['EDAD'].agg(['mean', 'std', 'count']).round(1)
    for platform, stats in age_by_platform.iterrows():
        print(f"• {platform}: {stats['mean']:.1f} ± {stats['std']:.1f} años (n={stats['count']:,})")

def analisis_diversidad_departamentos(df):
    """
    Analiza la diversidad de plataformas por departamento
    """
    print("\nAnalizando diversidad de plataformas por departamento...")
    
    # Seleccionar top 10 departamentos
    top_10_deptos = df['DEPARTAMENTO'].value_counts().head(10).index
    df_top_deptos = df[df['DEPARTAMENTO'].isin(top_10_deptos)]
    
    # Calcular índice de diversidad
    diversity_data = []
    for dept in df_top_deptos['DEPARTAMENTO'].unique():
        dept_data = df_top_deptos[df_top_deptos['DEPARTAMENTO'] == dept]
        platform_counts = dept_data['PLATAFORMA_EDUCATIVA'].value_counts()
        # Calcular índice de Simpson (diversidad)
        total = len(dept_data)
        simpson_index = sum([(count/total)**2 for count in platform_counts])
        diversity_index = 1 - simpson_index  # Más alto = más diverso
        diversity_data.append({'Departamento': dept, 'Diversidad': diversity_index})

    diversity_df = pd.DataFrame(diversity_data).sort_values('Diversidad')
    
    # Crear gráfico
    plt.figure(figsize=(10, 6))
    sns.barplot(data=diversity_df, x='Diversidad', y='Departamento', palette="viridis")
    plt.title('Índice de Diversidad de Plataformas por Departamento', fontsize=14, fontweight='bold')
    plt.xlabel('Índice de Diversidad (0-1)', fontsize=12)
    
    # Guardar gráfico
    plt.tight_layout()
    plt.savefig('graficos/diversidad_departamentos.png', dpi=300)
    plt.close()
    
    print("Gráfico guardado: graficos/diversidad_departamentos.png")
    
    # Mostrar top 3 departamentos con mayor diversidad
    print(f"\n🌍 DEPARTAMENTOS CON MAYOR DIVERSIDAD DE PLATAFORMAS:")
    for _, row in diversity_df.tail(3).iterrows():
        print(f"• {row['Departamento']}: {row['Diversidad']:.3f}")
    
    return diversity_df

def crear_mapa_beneficiarios(df):
    """
    Crea mapas interactivos con Folium mostrando la distribución por departamento
    """
    print("\nGenerando mapas interactivos de beneficiarios por departamento...")
    
    # Análisis detallado por departamento y plataforma
    conteo_plataformas = df.groupby(['DEPARTAMENTO', 'PLATAFORMA_EDUCATIVA']).size().reset_index(name='cantidad')
    
    # Crear matriz de distribución por departamento
    distribucion_matriz = conteo_plataformas.pivot(index='DEPARTAMENTO', columns='PLATAFORMA_EDUCATIVA', values='cantidad').fillna(0)
    distribucion_matriz['TOTAL'] = distribucion_matriz.sum(axis=1)
    
    # Calcular porcentajes por departamento
    distribucion_porcentajes = distribucion_matriz.div(distribucion_matriz['TOTAL'], axis=0) * 100
    distribucion_porcentajes = distribucion_porcentajes.drop('TOTAL', axis=1)
    
    # Colores más claros y visibles
    colores_claros = {
        'COURSERA': '#FFA500',      # Naranja brillante
        'PLATZI': '#32CD32',        # Verde lima
        'DATACAMP': '#4169E1',      # Azul real
        'MICROSOFT AZURE': '#FF6347', # Tomate
        'EDX': '#9370DB',           # Violeta medio
        'UDACITY': '#CD853F'        # Marrón arena
    }
    
    # Función para asignar color según plataforma dominante
    def crear_color_dominante(departamento):
        """Asigna color de la plataforma dominante"""
        if departamento not in distribucion_porcentajes.index:
            return '#f0f0f0'  # Gris muy claro
            
        dept_data = distribucion_porcentajes.loc[departamento]
        plataforma_dominante = dept_data.idxmax()
        porcentaje_dominante = dept_data.max()
        
        color_base = colores_claros.get(plataforma_dominante, '#808080')
        
        # Hacer el color más claro si la dominancia no es muy alta
        if porcentaje_dominante < 50:
            return f"{color_base}80"  # Añadir transparencia
        else:
            return color_base
    
    # Función para tooltip
    def crear_resumen_tooltip(departamento):
        """Crea un resumen conciso para el tooltip"""
        if departamento not in distribucion_matriz.index:
            return "Sin datos", 0
            
        total = int(distribucion_matriz.loc[departamento, 'TOTAL'])
        dept_data = distribucion_porcentajes.loc[departamento]
        
        # Solo top 3 plataformas para tooltip
        top_3 = dept_data.nlargest(3)
        
        resumen_parts = []
        for plataforma, porcentaje in top_3.items():
            if porcentaje >= 5:  # Solo mostrar si tiene al menos 5%
                resumen_parts.append(f"{plataforma}: {porcentaje:.0f}%")
                
        resumen = " | ".join(resumen_parts[:2])  # Solo top 2 para ser más corto
        if len(top_3) > 2 and top_3.iloc[2] >= 5:
            resumen += " | +"
            
        return resumen, total
    
    try:
        # Cargar GeoJSON
        url_geojson = "https://raw.githubusercontent.com/Emma-Ok/BootcampTalentoTech/main/colombia.geo.json"
        response = requests.get(url_geojson)
        if response.status_code != 200:
            print(f"Error al cargar GeoJSON: Código {response.status_code}")
            print("Descargando copia local del archivo...")
            
            # Intentar guardar una copia local del GeoJSON
            with open('colombia.geo.json', 'w', encoding='utf-8') as f:
                f.write(response.text)
                
            # Cargar desde archivo local
            with open('colombia.geo.json', 'r', encoding='utf-8') as f:
                geojson_data = json.load(f)
        else:
            geojson_data = response.json()
            
            # Guardar una copia local del GeoJSON
            with open('colombia.geo.json', 'w', encoding='utf-8') as f:
                json.dump(geojson_data, f, ensure_ascii=False)
                
        print("✅ GeoJSON cargado exitosamente")
        
        # Crear mapa con colores dominantes
        mapa_dominante = folium.Map(location=[4.5709, -74.2973], zoom_start=6)
        
        # Clonar GeoJSON para no modificar el original
        geojson_dominante = json.loads(json.dumps(geojson_data))
        
        for feature in geojson_dominante['features']:
            nombre_dpto = feature['properties']['NOMBRE_DPT'].upper()
            if nombre_dpto == 'SANTAFE DE BOGOTA D.C':
                nombre_dpto = 'BOGOTA'
                
            resumen, total = crear_resumen_tooltip(nombre_dpto)
            
            feature['properties']['total_participantes'] = total
            feature['properties']['distribucion_resumen'] = resumen
            feature['properties']['color_dominante'] = crear_color_dominante(nombre_dpto)
            
            # Información de la plataforma dominante
            if nombre_dpto in distribucion_porcentajes.index:
                dept_data = distribucion_porcentajes.loc[nombre_dpto]
                feature['properties']['plataforma_principal'] = dept_data.idxmax()
                feature['properties']['porcentaje_principal'] = f"{dept_data.max():.0f}%"
            else:
                feature['properties']['plataforma_principal'] = 'Sin datos'
                feature['properties']['porcentaje_principal'] = '0%'
        
        # Agregar GeoJSON al mapa
        GeoJson(
            geojson_dominante,
            style_function=lambda feature: {
                'fillColor': feature['properties']['color_dominante'],
                'color': '#333333',  # Borde más oscuro para mejor contraste
                'weight': 1.5,
                'fillOpacity': 0.8
            },
            tooltip=GeoJsonTooltip(
                fields=['NOMBRE_DPT', 'total_participantes', 'plataforma_principal', 'porcentaje_principal'],
                aliases=['Departamento:', 'Total beneficiarios:', 'Plataforma principal:', 'Porcentaje:'],
                localize=True,
                style=("background-color: white; color: #333333; font-family: arial; font-size: 11px; padding: 6px; border-radius: 3px; max-width: 250px;")
            )
        ).add_to(mapa_dominante)
        
        # Leyenda para el mapa
        legend_html = '''
        <div style="position: fixed;
                    bottom: 50px; left: 50px; width: 280px; height: 200px;
                    background-color: white; border:2px solid #333; z-index:9999;
                    font-size:12px; padding: 15px; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.2);">
        <p><b>🎓 Plataforma Dominante por Departamento</b></p>
        <p style="font-size:10px; margin-bottom:8px; color:#666;">Color = Plataforma con mayor participación</p>
        '''
        
        # Añadir cada plataforma con su color a la leyenda
        for plat, color in colores_claros.items():
            legend_html += f'<p><span style="color:{color}; font-size:16px;">⬤</span> {plat}</p>'
            
        legend_html += f'''
        <hr style="margin:8px 0;">
        <p style="font-size:10px; color:#666;">Total: {len(df):,} beneficiarios</p>
        </div>
        '''
        
        mapa_dominante.get_root().html.add_child(folium.Element(legend_html))
        
        # Guardar mapa
        mapa_dominante.save('graficos/mapa_plataformas_dominantes.html')
        print("Mapa guardado: graficos/mapa_plataformas_dominantes.html")
        
        # Mostrar estadísticas de los principales departamentos
        top_departamentos = distribucion_matriz.nlargest(5, 'TOTAL')
        print(f"\n🏆 TOP 5 DEPARTAMENTOS CON MAYOR PARTICIPACIÓN:")
        for dept in top_departamentos.index:
            total = int(top_departamentos.loc[dept, 'TOTAL'])
            dept_dist = distribucion_porcentajes.loc[dept]
            principal = dept_dist.idxmax()
            print(f"• {dept}: {total:,} usuarios, Principal: {principal} ({dept_dist.max():.0f}%)")
        
    except Exception as e:
        print(f"Error al crear el mapa: {e}")

def crear_carpeta_graficos():
    """
    Crea la carpeta para almacenar los gráficos generados
    """
    # Crear carpeta de gráficos si no existe
    if not os.path.exists('graficos'):
        os.makedirs('graficos')
        print("Carpeta 'graficos' creada")

# Nuevas funciones del 2do Objetivo Específico

def analisis_distribucion_plataformas(df):
    """
    Realiza un análisis completo de la distribución por plataformas
    """
    print("\nGenerando análisis de distribución por plataformas...")
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Análisis de Participación por Plataforma Educativa', fontsize=16, fontweight='bold')

    # 1.1 Participación general por plataforma
    ax1 = axes[0, 0]
    plataforma_counts = df['PLATAFORMA_EDUCATIVA'].value_counts()
    colors = sns.color_palette("viridis", len(plataforma_counts))
    bars = ax1.bar(range(len(plataforma_counts)), plataforma_counts.values, color=colors)
    ax1.set_xticks(range(len(plataforma_counts)))
    ax1.set_xticklabels(plataforma_counts.index, rotation=45, ha='right')
    ax1.set_title('Distribución General por Plataforma', fontweight='bold')
    ax1.set_ylabel('Número de Beneficiarios')

    # Agregar valores en las barras
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                f'{int(height):,}', ha='center', va='bottom')

    # 1.2 Participación por género y plataforma
    ax2 = axes[0, 1]
    gender_platform = pd.crosstab(df['PLATAFORMA_EDUCATIVA'], df['GENERO'])
    gender_platform.plot(kind='bar', ax=ax2, color=['#FF6B9D', '#4ECDC4'])
    ax2.set_title('Distribución por Género y Plataforma', fontweight='bold')
    ax2.set_ylabel('Número de Beneficiarios')
    ax2.legend(title='Género')
    ax2.tick_params(axis='x', rotation=45)

    # 1.3 Top 10 departamentos por plataforma
    ax3 = axes[1, 0]
    top_deptos = df['DEPARTAMENTO'].value_counts().head(10)
    sns.barplot(x=top_deptos.values, y=top_deptos.index, ax=ax3, palette="plasma")
    ax3.set_title('Top 10 Departamentos con Mayor Participación', fontweight='bold')
    ax3.set_xlabel('Número de Beneficiarios')

    # 1.4 Heatmap de plataformas por región (top 10 deptos)
    ax4 = axes[1, 1]
    top_10_deptos = df['DEPARTAMENTO'].value_counts().head(10).index
    df_top_deptos = df[df['DEPARTAMENTO'].isin(top_10_deptos)]
    heatmap_data = pd.crosstab(df_top_deptos['DEPARTAMENTO'], df_top_deptos['PLATAFORMA_EDUCATIVA'])
    sns.heatmap(heatmap_data, annot=True, fmt='d', cmap='YlOrRd', ax=ax4)
    ax4.set_title('Matriz: Departamentos vs Plataformas', fontweight='bold')
    ax4.set_xlabel('Plataforma Educativa')

    plt.tight_layout()
    plt.savefig('graficos/analisis_distribucion_plataformas.png', dpi=300)
    plt.close()
    
    print("Gráfico guardado: graficos/analisis_distribucion_plataformas.png")

def analisis_demografico_detallado(df):
    """
    Realiza un análisis demográfico detallado
    """
    print("\nGenerando análisis demográfico detallado...")
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Análisis Demográfico Detallado', fontsize=16, fontweight='bold')

    # 2.1 Distribución de edades por plataforma
    ax1 = axes[0, 0]
    sns.boxplot(data=df, x='PLATAFORMA_EDUCATIVA', y='EDAD', ax=ax1, palette="Set2")
    ax1.set_title('Distribución de Edad por Plataforma', fontweight='bold')
    ax1.tick_params(axis='x', rotation=45)
    ax1.set_ylabel('Edad (años)')

    # 2.2 Densidad de edad por plataforma
    ax2 = axes[0, 1]
    for platform in df['PLATAFORMA_EDUCATIVA'].unique():
        subset = df[df['PLATAFORMA_EDUCATIVA'] == platform]
        sns.kdeplot(subset['EDAD'], label=platform, ax=ax2, alpha=0.7)
    ax2.set_title('Densidad de Edad por Plataforma', fontweight='bold')
    ax2.set_xlabel('Edad (años)')
    ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    # 2.3 Violin plot: Edad por género y plataforma
    ax3 = axes[1, 0]
    # Seleccionar top 4 plataformas para mejor visualización
    top_platforms = df['PLATAFORMA_EDUCATIVA'].value_counts().head(4).index
    df_top_platforms = df[df['PLATAFORMA_EDUCATIVA'].isin(top_platforms)]
    sns.violinplot(data=df_top_platforms, x='PLATAFORMA_EDUCATIVA', y='EDAD',
                hue='GENERO', ax=ax3, palette="muted", split=True)
    ax3.set_title('Distribución de Edad por Género (Top 4 Plataformas)', fontweight='bold')
    ax3.tick_params(axis='x', rotation=45)

    # 2.4 Promedio de edad por departamento (top 10)
    ax4 = axes[1, 1]
    top_10_deptos = df['DEPARTAMENTO'].value_counts().head(10).index
    df_top_deptos = df[df['DEPARTAMENTO'].isin(top_10_deptos)]
    age_by_dept = df_top_deptos.groupby('DEPARTAMENTO')['EDAD'].mean().sort_values(ascending=True)
    sns.barplot(x=age_by_dept.values, y=age_by_dept.index, ax=ax4, palette="coolwarm")
    ax4.set_title('Edad Promedio por Departamento (Top 10)', fontweight='bold')
    ax4.set_xlabel('Edad Promedio (años)')

    plt.tight_layout()
    plt.savefig('graficos/analisis_demografico_detallado.png', dpi=300)
    plt.close()
    
    print("Gráfico guardado: graficos/analisis_demografico_detallado.png")

def analisis_patrones_preferencias(df):
    """
    Analiza patrones y preferencias de uso de plataformas
    """
    print("\nGenerando análisis de patrones y preferencias...")
    
    # Confirmar que tenemos el grupo etario
    if 'GRUPO_ETARIO' not in df.columns:
        df = crear_grupos_etarios(df)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Patrones de Acceso y Preferencias', fontsize=16, fontweight='bold')

    # 3.1 Preferencias por grupo de edad
    ax1 = axes[0, 0]
    age_platform = pd.crosstab(df['GRUPO_ETARIO'], df['PLATAFORMA_EDUCATIVA'], normalize='index') * 100
    sns.heatmap(age_platform, annot=True, fmt='.1f', cmap='Blues', ax=ax1)
    ax1.set_title('Preferencias de Plataforma por Grupo de Edad (%)', fontweight='bold')
    ax1.set_ylabel('Grupo de Edad')

    # 3.2 Análisis regional por plataforma
    ax2 = axes[0, 1]
    region_platform = pd.crosstab(df['DEPARTAMENTO'], df['PLATAFORMA_EDUCATIVA'])
    region_platform.plot(kind='bar', ax=ax2, stacked=True, colormap='tab10')
    ax2.set_title('Distribución de Plataformas por Departamento', fontweight='bold')
    ax2.set_ylabel('Número de Beneficiarios')
    ax2.legend(title='Plataforma', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax2.tick_params(axis='x', rotation=45)

    # 3.3 Índice de diversidad por departamento
    ax3 = axes[1, 0]
    top_10_deptos = df['DEPARTAMENTO'].value_counts().head(10).index
    df_top_deptos = df[df['DEPARTAMENTO'].isin(top_10_deptos)]
    
    diversity_data = []
    for dept in df_top_deptos['DEPARTAMENTO'].unique():
        dept_data = df_top_deptos[df_top_deptos['DEPARTAMENTO'] == dept]
        platform_counts = dept_data['PLATAFORMA_EDUCATIVA'].value_counts()
        # Calcular índice de Simpson (diversidad)
        total = len(dept_data)
        simpson_index = sum([(count/total)**2 for count in platform_counts])
        diversity_index = 1 - simpson_index  # Más alto = más diverso
        diversity_data.append({'Departamento': dept, 'Diversidad': diversity_index})

    diversity_df = pd.DataFrame(diversity_data).sort_values('Diversidad')
    sns.barplot(data=diversity_df, x='Diversidad', y='Departamento', ax=ax3, palette="viridis")
    ax3.set_title('Índice de Diversidad de Plataformas por Departamento', fontweight='bold')
    ax3.set_xlabel('Índice de Diversidad (0-1)')

    # 3.4 Concentración de usuarios por plataforma y género
    ax4 = axes[1, 1]
    gender_concentration = df.groupby(['PLATAFORMA_EDUCATIVA', 'GENERO']).size().unstack(fill_value=0)
    gender_concentration_pct = gender_concentration.div(gender_concentration.sum(axis=1), axis=0) * 100
    sns.heatmap(gender_concentration_pct, annot=True, fmt='.1f', cmap='RdYlBu', ax=ax4)
    ax4.set_title('Concentración de Género por Plataforma (%)', fontweight='bold')

    plt.tight_layout()
    plt.savefig('graficos/analisis_patrones_preferencias.png', dpi=300)
    plt.close()
    
    print("Gráfico guardado: graficos/analisis_patrones_preferencias.png")

def crear_mapa_proporcional(df):
    """
    Crea un mapa proporcional de beneficiarios
    """
    print("\nGenerando mapa con visualización proporcional...")
    
    # 1. Análisis detallado por departamento y plataforma
    conteo_plataformas = df.groupby(['DEPARTAMENTO', 'PLATAFORMA_EDUCATIVA']).size().reset_index(name='cantidad')
    df_departamentos = df.groupby('DEPARTAMENTO').size().reset_index(name='total_participantes')

    print("📊 Preparando datos para visualización proporcional...")

    # 2. Crear matriz de distribución por departamento
    distribucion_matriz = conteo_plataformas.pivot(index='DEPARTAMENTO', columns='PLATAFORMA_EDUCATIVA', values='cantidad').fillna(0)
    distribucion_matriz['TOTAL'] = distribucion_matriz.sum(axis=1)

    # Calcular porcentajes por departamento
    distribucion_porcentajes = distribucion_matriz.div(distribucion_matriz['TOTAL'], axis=0) * 100
    distribucion_porcentajes = distribucion_porcentajes.drop('TOTAL', axis=1)

    print("📈 Distribución por departamento (Top 5):")
    top_5_display = distribucion_matriz.sort_values('TOTAL', ascending=False).head(5)
    for dept in top_5_display.index:
        print(f"  {dept}: {int(top_5_display.loc[dept, 'TOTAL'])} participantes")

    # 3. Colores más claros y visibles
    colores_claros = {
        'COURSERA': '#FFA500',      # Naranja brillante
        'PLATZI': '#32CD32',        # Verde lima
        'DATACAMP': '#4169E1',      # Azul real
        'MICROSOFT AZURE': '#FF6347', # Tomate
        'EDX': '#9370DB',           # Violeta medio
        'UDACITY': '#CD853F'        # Marrón arena
    }

    # 5. Función para crear color sólido basado en la plataforma dominante (más claro)
    def crear_color_dominante_claro(departamento):
        """
        Asigna color de la plataforma dominante pero más claro y visible
        """
        if departamento not in distribucion_porcentajes.index:
            return '#f0f0f0'  # Gris muy claro

        dept_data = distribucion_porcentajes.loc[departamento]
        plataforma_dominante = dept_data.idxmax()
        porcentaje_dominante = dept_data.max()

        color_base = colores_claros.get(plataforma_dominante, '#808080')

        # Hacer el color más claro si la dominancia no es muy alta
        if porcentaje_dominante < 50:
            # Mezclar con blanco para hacer más claro
            return f"{color_base}80"  # Añadir transparencia
        else:
            return color_base

    # 6. Crear información resumida para tooltips (más corta)
    def crear_resumen_tooltip(departamento):
        """
        Crea un resumen conciso para el tooltip
        """
        if departamento not in distribucion_matriz.index:
            return "Sin datos", 0

        total = int(distribucion_matriz.loc[departamento, 'TOTAL'])
        dept_data = distribucion_porcentajes.loc[departamento]

        # Solo top 3 plataformas para mantener tooltip corto
        top_3 = dept_data.nlargest(3)

        resumen_parts = []
        for plataforma, porcentaje in top_3.items():
            if porcentaje >= 5:  # Solo mostrar si tiene al menos 5%
                cantidad = int(distribucion_matriz.loc[departamento, plataforma])
                resumen_parts.append(f"{plataforma}: {porcentaje:.0f}%")

        resumen = " | ".join(resumen_parts[:2])  # Solo top 2 para ser más corto
        if len(top_3) > 2 and top_3.iloc[2] >= 5:
            resumen += " | +"

        return resumen, total

    # 7. Cargar GeoJSON
    print("🌍 Cargando archivo GeoJSON...")
    try:
        url_geojson = "https://raw.githubusercontent.com/Emma-Ok/BootcampTalentoTech/main/colombia.geo.json"
        response = requests.get(url_geojson)
        geojson_data = response.json()
        print("✅ GeoJSON cargado exitosamente")

        # MAPA 2: GRÁFICOS DE BARRAS SUPERPUESTOS
        print("📊 Creando Mapa: Con Gráficos de Barras...")

        mapa_barras = folium.Map(location=[4.5709, -74.2973], zoom_start=6)

        # Agregar el mapa base con color neutral
        GeoJson(
            geojson_data,
            style_function=lambda feature: {
                'fillColor': '#f8f8f8',  # Gris muy claro como base
                'color': '#666666',
                'weight': 1,
                'fillOpacity': 0.3
            }
        ).add_to(mapa_barras)

        # Agregar marcadores con gráficos de barras para departamentos principales
        top_departamentos = distribucion_matriz.nlargest(15, 'TOTAL')  # Top 15 departamentos

        # Coordenadas aproximadas de capitales de departamento (para centrar los gráficos)
        coordenadas_deptos = {
            'BOGOTA': [4.7110, -74.0721],
            'ANTIOQUIA': [6.2442, -75.5812],
            'VALLE DEL CAUCA': [3.4516, -76.5320],
            'CUNDINAMARCA': [4.7110, -74.0721],
            'SANTANDER': [7.1193, -73.1227],
            'ATLANTICO': [10.9639, -74.7964],
            'CALDAS': [5.0703, -75.5138],
            'RISARALDA': [4.8087, -75.6906],
            'BOLIVAR': [10.3932, -75.4832],
            'CAUCA': [2.4448, -76.6147],
            'NORTE DE SANTANDER': [7.8939, -72.5078],
            'HUILA': [1.9344, -75.5277],
            'TOLIMA': [4.4389, -75.2322],
            'BOYACA': [5.4539, -73.3616],
            'NARIÑO': [1.2136, -77.2811]
        }

        for dept in top_departamentos.index[:10]:  # Solo top 10 para no saturar
            if dept in coordenadas_deptos:
                coords = coordenadas_deptos[dept]
                dept_data = distribucion_porcentajes.loc[dept]
                total_users = int(distribucion_matriz.loc[dept, 'TOTAL'])

                # Crear HTML para gráfico de barras mini
                top_3_plat = dept_data.nlargest(3)

                barras_html = f"""
                <div style="background: white; padding: 8px; border-radius: 5px; border: 2px solid #333; min-width: 120px;">
                    <b style="font-size: 10px;">{dept}</b><br>
                    <span style="font-size: 9px; color: #666;">{total_users} usuarios</span><br>
                """

                for plataforma, porcentaje in top_3_plat.items():
                    if porcentaje >= 3:
                        color = colores_claros.get(plataforma, '#808080')
                        ancho = int(porcentaje * 0.8)  # Escalar para que quepa
                        barras_html += f"""
                        <div style="margin: 2px 0;">
                            <span style="font-size: 8px; display: inline-block; width: 40px;">{plataforma[:4]}</span>
                            <div style="display: inline-block; background: {color}; width: {ancho}px; height: 8px; margin-left: 2px;"></div>
                            <span style="font-size: 8px; margin-left: 2px;">{porcentaje:.0f}%</span>
                        </div>
                        """

                barras_html += "</div>"

                # Agregar marcador con el gráfico
                folium.Marker(
                    location=coords,
                    popup=folium.Popup(barras_html, max_width=200),
                    icon=folium.DivIcon(
                        html=f"""<div style="background: rgba(255,255,255,0.9); border: 1px solid #333; border-radius: 3px; padding: 2px; font-size: 8px; font-weight: bold; text-align: center;">{dept[:8]}<br>{total_users}</div>""",
                        icon_size=(60, 25),
                        icon_anchor=(30, 12)
                    )
                ).add_to(mapa_barras)

        # Leyendas mejoradas
        legend_barras = '''
        <div style="position: fixed;
                    bottom: 50px; left: 50px; width: 250px; height: 160px;
                    background-color: white; border:2px solid #333; z-index:9999;
                    font-size:12px; padding: 15px; border-radius: 8px; box-shadow: 0 2px 10px rgba(0,0,0,0.2);">
        <p><b>📊 Distribución Detallada</b></p>
        <p style="font-size:10px; margin-bottom:8px; color:#666;">Haz clic en los marcadores para ver gráficos de barras</p>
        <p style="font-size:10px; color:#666;">• Marcadores = Top 10 departamentos</p>
        <p style="font-size:10px; color:#666;">• Barras = Proporción por plataforma</p>
        <p style="font-size:10px; color:#666;">• Números = Total de usuarios</p>
        </div>
        '''

        mapa_barras.get_root().html.add_child(folium.Element(legend_barras))

        # Guardar mapa
        mapa_barras.save('graficos/mapa_barras_departamentos.html')
        print("Mapa guardado: graficos/mapa_barras_departamentos.html")

        # Estadísticas finales
        print(f"\n🏆 TOP 5 DEPARTAMENTOS CON MEJOR DISTRIBUCIÓN:")
        for dept in top_departamentos.head(5).index:
            total = int(top_departamentos.loc[dept, 'TOTAL'])
            dept_dist = distribucion_porcentajes.loc[dept]
            principal = dept_dist.idxmax()
            print(f"• {dept}: {total:,} usuarios, Principal: {principal} ({dept_dist.max():.0f}%)")

    except Exception as e:
        print(f"❌ Error al crear el mapa proporcional: {e}")

def main():
    """
    Función principal que ejecuta todo el análisis
    """
    print("==== ANÁLISIS DE BENEFICIARIOS Y PLATAFORMAS EDUCATIVAS ====")
    
    # Crear carpeta de gráficos
    crear_carpeta_graficos()
    
    # Cargar y preparar datos
    df = cargar_datos()
    df = crear_grupos_etarios(df)
    
    # Análisis general del primer objetivo
    grafico_departamentos(df)
    grafico_genero(df)
    grafico_edad(df)
    grafico_plataformas(df)
    grafico_plataformas_genero(df)
    grafico_plataformas_edad(df)
    grafico_grupos_plataformas_barras(df)
    
    # Análisis de diversidad
    diversity_df = analisis_diversidad_departamentos(df)
    
    # Análisis del segundo objetivo
    analisis_distribucion_plataformas(df)
    analisis_demografico_detallado(df)
    analisis_patrones_preferencias(df)
    crear_mapa_proporcional(df)
    
    # Estadísticas resumen
    analisis_estadistico(df)
    
    # Crear mapas interactivos
    crear_mapa_beneficiarios(df)
    
    print("\n==== ANÁLISIS COMPLETADO ====")
    print("Todos los gráficos han sido guardados en la carpeta 'graficos'")
    print("Para ver los mapas interactivos, abra los archivos HTML en su navegador:")
    print("  $BROWSER graficos/mapa_plataformas_dominantes.html")
    print("  $BROWSER graficos/mapa_barras_departamentos.html")

if __name__ == "__main__":
    main()

==== ANÁLISIS DE BENEFICIARIOS Y PLATAFORMAS EDUCATIVAS ====
Carpeta 'graficos' creada
Cargando datos de beneficiarios...
Dataset cargado: 4895 registros con 5 variables

Primeras 5 filas del DataFrame:
      DEPARTAMENTO  MUNICIPIO     GENERO  EDAD PLATAFORMA_EDUCATIVA
0     CUNDINAMARCA  ZIPAQUIRA  MASCULINO    31             COURSERA
1           NARIÑO      PASTO  MASCULINO    40             DATACAMP
2  VALLE DEL CAUCA      YUMBO  MASCULINO    28               PLATZI
3        ANTIOQUIA   SABANETA   FEMENINO    35      MICROSOFT AZURE
4        ANTIOQUIA   SABANETA  MASCULINO    35               PLATZI
Datos procesados: 4895 registros válidos

Generando gráfico de beneficiarios por departamento...
Gráfico guardado: graficos/beneficiarios_por_departamento.png

Generando gráfico de distribución por género...
Gráfico guardado: graficos/beneficiarios_por_departamento.png

Generando gráfico de distribución por género...
Gráfico guardado: graficos/distribucion_genero.png

Generando histogra

<Figure size 1120x560 with 0 Axes>